# Initialization

In [3]:
# import
import aiohttp
import pkuseg
import pymongo
import zhconv
from types import TracebackType
from typing import Dict, Optional, Type
from aip import AipNlp
from motor import motor_asyncio
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

    
# CONFIG
if platform.system() == 'Windows':
    # CODE_PATH
    CODE_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/code')
    DATA_PATH = os.path.join(os.environ['onedrive'], 'STN/10-FirstYear/data')
    BERT_PATH = os.path.join(os.environ['onedrive'], 'Data/BERT')
    ERNIE_PATH = os.path.join(os.environ['onedrive'], 'Data/baidu-paddle/LARK/ERNIE')
    ERNIE_LAC_PATH=os.path.join(os.environ['onedrive'], 'Data/baidu-paddle/models/PaddleNLP/lexical_analysis')
    %cd {CODE_PATH}
elif platform.system() == 'Linux':
    # CODE_PATH
    CODE_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/code'
    DATA_PATH = '/mnt/c/Users/rossz/OneDrive/STN/10-FirstYear/data'
    %cd {CODE_PATH}

# Baidu NLP
def get_token(accounts):
    '''get API tokens'''
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials"
    for account in accounts:
        # client_id 为官网获取的AK， client_secret 为官网获取的SK
        param = {'client_id': account['apiKey'],
                 'client_secret': account['secretKey']}
        resp = requests.get(url, param).json()
        yield resp['access_token']
        
accounts = [{'appId': '16105374', # ross.zhu@outlook.com
            'apiKey': 'zShobECEYX2B2CNrcz3QsM3I',
            'secretKey': 'xM8g1lkhTd8b7DYGv2H9j7g2hvthPGE5'},
           {'appId': '16218655', # 18868819096
            'apiKey': 'AG2ygGZbCPqDW25ZC6FinQRy',
            'secretKey': 'pcx8pFwjVhD0oGDsdm58bzt7Zm9zFpWk'},
           {'appId': '16226200', # 13566633705
            'apiKey': 'ymmGiPQoAoSF2KhcGVKNZEVj',
            'secretKey': '78oQXsRtdmRcINEGtP3aP2y4sKA7HerU'},
           {'appId': '16226243', # 13606586786
            'apiKey': 'jIy3T25rYR1Kj9BlCKPcHbYe',
            'secretKey': 'jORNEeu7Qnl6L1Hr07NomigmsUtXlloL'},
           {'appId': '16230746', # 李锐
            'apiKey': 't6sWsoIYinoGBgwbLiCopQHR',
            'secretKey': 'evq4UygxQ555LXzVzaDWQS2TfGNUHrYf'},
           {'appId': '16307961', # 俞嘉炜
            'apiKey': 'TKIEhGe1fIY89VlYm8yjuDZW',
            'secretKey': '6kDBxZnGZEsK4y9ntg94VOBSIjLS0UQs'},
           {'appId': '16305932', # 芳菲
            'apiKey': 'Y7NjBe2sqhfIjSIaB51wvlHH',
            'secretKey': 'lqLW2XeSj0xBvCbUASh0E2wuMmW6e4ox'},
           {'appId': '16305931', # 小毫生
            'apiKey': 'D2GGEWS80KsU9mtft1WnuR0N',
            'secretKey': '8FbvLeGD5aXLL6bpKSEZoG7CRt1Q9oqo'},
           {'appId': '16306399', # 忻小宇
            'apiKey': 'E89qU1OMrwvyanH7xixkFVrR',
            'secretKey': 'ljPLgCtzOKLZuG56DPX9hvrVFHRsWHqf'},
           {'appId': '16306475', # 杨老板
            'apiKey': 'yehP04KXoWADTGUyLF8WK0hN',
            'secretKey': 'jfp3wxqupbWfEoNi7L316LdfbUmDmRUo'},
           ]
tokens = list(get_token(accounts))
baidu_clients = [AipNlp(**account) for account in accounts] 

# MongoDB-pymongo
pymongo_client = MongoClient('localhost', 27018)
db = pymongo_client.exchange
coll_emb_bd = db.emb_bd
coll_vocab_without_emb = db.vocab_without_emb
coll_vocab_to_request = db.vocab_to_request
coll_vocab_pos = db.vocab_pos
coll_vocab_ne = db.vocab_ne
coll_text_seg_bd = db.text_seg_bd
coll_text_seg_pku = db.text_seg_pku
coll_title_seg_bd = db.title_seg_bd
coll_title_seg_pku = db.title_seg_pku

coll_test = db.test

# MongoDB-motor
# loop=asyncio.new_event_loop()
# asyncio.set_event_loop(loop)
# motor_client = motor_asyncio.AsyncIOMotorClient('localhost', 27018, io_loop=loop)
# db = motor_client.exchange
# coll_mt_embedding_bd = db.embedding_bd
# coll_mt_vocab_without_emb = db.vocab_without_emb
# coll_mt_vocab_to_request = db.vocab_to_request
# coll_mt_vocab_pos = db.vocab_pos
# coll_mt_vocab_ne = db.vocab_ne
# coll_mt_text_seg_bd = db.text_seg_bd
# coll_mt_title_seg_bd = db.title_seg_bd
# coll_mt_test = db.test

C:\Users\rossz\OneDrive\STN\10-FirstYear\code


# Create `pkl` from R

In [ ]:
# %%time
# cmt_text/title is created from R
cmt = dt.fread('../data/cmt.csv', sep='\t', encoding='utf-8')
cmt_text = cmt[:, 'text'].to_list()[0]
sv('cmt_text')
cmt_title = cmt[:, 'title'].to_list()[0]
sv('cmt_title')

# every col except {'title', 'text'}
cmt_without_title_and_text = cmt[:, {'id': f.id, 'user_id': f['user.id'],
                     'created_at': f['created.at'],
                     'source': f.source,
                     'comment_id': f['comment.id'],
                     'retweet_status_id': f['retweet.status.id'],
                     'lastcrawl': f.lastcrawl}]
cmt_without_title_and_text.to_csv('../data/cmt_no_text.csv')

# Clean `cmt`

## function

In [87]:
# clean cmt_text
def clean_text(text: str):
    import re
    import zhconv
    # text wrapped in <p>
    p = re.compile(r'<p>(.*?)</p>')
    text = p.sub(r' \1 ', text)

    # emoji
    p = re.compile(r'<img.*?imedao.*?title="\[(.+?)\]".*?alt="\[(.+?)\]".*?/>')
    text = p.sub(r' \1 ', text)
    
    # @someone
    # remove everything
    p = re.compile(r'<a href[^#$]*?>(@.+?)</a>')
    text = p.sub(r' ', text)
    
    # $stock$
    p = re.compile(r'<a href[^@#]*?>\$(.+?)[(（](\w*?)(\d*?)[)）]\$</a>')
    text = p.sub(r' \1 \2 \3 ', text)
    
    # #摇一摇# xueqiu campain
    p = re.compile(r'<a href[^@$]*?>(#.+?#)</a>')
    text = p.sub(r' \1 ', text)
    
    # user images
    p = re.compile(r'<img.*?/>')
    text = p.sub(r' ', text)
    
    # other hyperlinks
    p = re.compile(r'<a href[^@$#]+?</a>')
    text = p.sub(r' ', text)
    
    # other html tags
    p = re.compile(r'(</?br? ?/?>|&nbsp;|</?p>|<h\d>|</?strong>|&.+?dquo;|<p.+?>|&quot;)')
    text = p.sub(r' ', text)
    
    # replace all duplicated space
    p = re.compile(r'\s+')
    text = p.sub(r' ', text).strip()
    
    # all english letter to lower
    text = text.lower()
    # traditional to simplified
    text = zhconv.convert(text, 'zh-cn')
    
    return text

## clean title

In [89]:
%%time
cmt_title = ld('cmt_title')
with Pool() as pool:
    cmt_title_clean = list(tqdm(pool.imap(clean_text, cmt_title, chunksize=1024), total=len(cmt_title)))
sv('cmt_title_clean')

## clean text

In [91]:
%%time
cmt_text = ld('cmt_text')

Wall time: 1min 31s


In [92]:
with Pool() as pool:
    cmt_text_clean = list(tqdm(pool.imap(clean_text, cmt_text, chunksize=1024), total=len(cmt_text)))

100%|██████████████████████████████████████████████████████████████████████████████| 44405159/44405159 [08:28<00:00, 87257.36it/s]


In [93]:
%%time
sv('cmt_text_clean')

Wall time: 4min 28s


# Segmentation

## PKU Seg

In [2]:
# %%time
def seg_pku(text, coll_cmt):
    segger = pkuseg.pkuseg(model_name='web')
    print(f'scanning requested docs...{Now()}')
    vocab_to_request = set(v['word'] for v in coll_vocab_to_request.find(projection={'word':1, '_id':0}))
    vocab_with_emb = set(v['word'] for v in coll_emb_bd.find(projection={'word':1, '_id':0}))
    vocab_without_emb = set(v['word'] for v in coll_vocab_without_emb.find(projection={'word':1, '_id':0}))
    vocab = vocab_to_request | vocab_with_emb | vocab_without_emb
    idx = set(doc['id'] for doc in coll_cmt.find(projection={'id':1, '_id':0}))
    
    # cut
    print(f'start cutting...{Now()}')
    for i, t in tqdm(enumerate(text), total=len(text)):
        if t != '' and i not in idx :
            segged = segger.cut(t)
            try:
                coll_cmt.insert_one({'id':i, 'text_seg':' '.join(segged)})
                for word in segged:
                    if word not in vocab:
                        coll_vocab_to_request.insert_one({'word': word})
            except DuplicateKeyError:
                pass
            except:
                print(sys.exc_info()[0])

In [9]:
%%time
# title
if 'cmt_title_clean' not in globals():
    print(f'loading cmt_title_clean...{Now()}')
    pickle.load(open('../data/cmt_title_clean.pkl', 'rb'))
seg_pku(cmt_title_clean, coll_title_seg_pku)

loading vocab_to_request...10:17:52
start cutting...10:18:06


100%|██████████████████████████████████████████████████████████████████████████████| 44405159/44405159 [13:53<00:00, 53289.56it/s]


Wall time: 14min 8s


In [3]:
%%time
# text
if 'cmt_text_clean' not in globals():
    print(f'loading cmt_text_clean...{Now()}')
    cmt_text_clean = pickle.load(open('../data/cmt_text_clean.pkl', 'rb'))
seg_pku(cmt_text_clean, coll_text_seg_pku)

loading cmt_text_clean...05:28:35
scanning requested docs...05:29:58
start cutting...05:32:31


100%|██████████████████████████████████████████████████████████████| 44405159/44405159 [00:14<00:00, 3052740.45it/s]


Wall time: 4min 12s


## Baidu API

### create requests

In [ ]:
%%time
def build_requests(cmt_path, cmt_coll):
    cmt_text = pickle.load(open(cmt_path, 'rb'))
    cmt_text = [(i, x) for i, x in enumerate(cmt_text)]

    segged_cmt_id = set(x['id'] for x in cmt_coll.find(projection={'id':1, '_id':0}))
    print(f'{len(segged_cmt_id)} docs have been segged.')
    
    request_cmt_id = set(range(len(cmt_text))) - segged_cmt_id
    request_cmt = [(i, doc) for i, doc in cmt_text if i in request_cmt_id]
    print(f'{len(request_cmt_id)} docs to reqeust.')
    
    vocab_to_request = set(x['word'] for x in coll_vocab_to_request.find(projection={'word':1, '_id':0}))
    print(f'{len(vocab_to_request)} words to request')
    
    return request_cmt, vocab_to_request

# request_cmt_title, request_cmt_id = build_requests('../data/cmt_title_clean.pkl', coll_title_seg_bd)
request_cmt_text, vocab_to_request = build_requests('../data/cmt_text_clean.pkl', coll_text_seg_bd)

### sequential

In [ ]:
def seg_bd(corpus, coll):
    # func to write coll_vocab
    def write_vocab(coll, iterable, coll_type='to_request', pos=None, ne=None):
        nonlocal count_duplicate, count_mongo_error
        try:
            with capture_output():
                for word in iterable:
                    if coll_type == 'to_request':
                        coll.insert_one({'word': word})
                    elif coll_type == 'pos':
                        coll.insert_one({'word': word, coll_type: pos[word]})
                    elif coll_type == 'ne':
                        coll.insert_one({'word': word, coll_type: ne[word]})
        except DuplicateKeyError:
            pass
        except:
            count_mongo_error += 1
            print(f'mongo error while inserting {coll.name}: {sys.exc_info()}')
        
    # build requests
    # if request is empty, don't send!
    id_last = set(x['id'] for x in coll.find(projection={'id':1, '_id':-1}))
    print(f'text already segged: {len(id_last)}')
    id_request = set([i for i, _ in corpus]) - id_last
    corpus = [(i, doc) for i, doc in corpus if i in id_request and doc != '']
    print(f'text to request: {len(corpus)}')
    
    # send request & process response
    count_all, last_count_all, count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0, 0
    ts = time.time()
    for idx, text in corpus:
        count_all += 1
        time.sleep(0.02)
        try:
            # if empty request, directly write to mongo
            request_text = text.encode('gbk', errors='ignore').decode('gbk')
            if request_text == '': # after re-encoding, request may be empty
                count_empty_input += 1
                coll.insert_one({'id': idx, 'text_seg': '', 'text_seg2': ''})
                count_success += 1
            # else, send request and get response
            else:
                response = client_bd.lexer(request_text)
                # process errors in response
                if 'error_code' in response:
                    if response['error_code'] == 18: # limit reached
                        count_limit_reached += 1
                    elif response['error_code'] == 282131: # input too long
                        count_input_too_long += 1
                    else:
                        print(f'other error code: request:"{request_text}", response:"{response}"')
                        count_other_error += 1
                else:
                    # process successful response
                    response = response['items']
                    vocab_coarse = [item['item'] for item in response]
                    vocab_granular = list(itertools.chain.from_iterable([item['basic_words'] for item in response]))
                    text_seg = ' '.join(vocab_coarse)
                    text_seg2 = ' '.join(vocab_granular)
                    pos = {item['item']: item['pos'] for item in response}
                    ne = {item['item']: item['ne'] for item in response}

                    # write coll_text_seg_bd
                    with capture_output():
                        coll.insert_one({'id': idx, 'text_seg': text_seg, 'text_seg2': text_seg2})
                        count_success += 1

                    # write coll_vocab_to_reqeust
                    write_vocab(coll_vocab_to_request, vocab_coarse)
                    write_vocab(coll_vocab_to_request, vocab_granular)
                    # write coll_vocab_pos
                    write_vocab(coll_vocab_pos, vocab_coarse, 'pos', pos=pos)
                    # write coll_vocab_ne
                    write_vocab(coll_vocab_ne, vocab_coarse, 'ne', ne=ne)
            
        except DuplicateKeyError:
            count_duplicate += 1
        except:
            print(f'unknow error: {sys.exc_info()}')
            count_other_error += 1
        finally:
            # log
            if count_all % 200 == 0:
                print(f'{count_all}/{len(corpus)}, {Now()} ({time.time()-ts:.1f}s / {(count_all-last_count_all)/(time.time()-ts):.1f}QPS)')
                print(f'success:{count_success}, limit-reached:{count_limit_reached}, empty-input-in-success:{count_empty_input}, dup:{count_duplicate}, input-too-long:{count_input_too_long}, other-baidu-error:{count_other_error}, other-mongo-error:{count_mongo_error}')
                print('---------------------')
                count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0
                ts = time.time()
                last_count_all = count_all
                      
# text/title
seg_bd(cmt_title_clean[40000000:], coll_title_seg_bd)

### async

In [ ]:
async def get_lex(text, token):
    base_url = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer'
    url_params = {'access_token': token,
                  'charset': 'UTF-8'}
    headers = {'Content-Type': 'application/json'}
    request_text = {'text': text}

    async with aiohttp.ClientSession() as session:
        async with session.request('POST', url=base_url, json=request_text, headers=headers, params=url_params) as resp:
            response = await resp.json()
            return response
        
async def seg_one(idx, text, token, coll_cmt, bucket, download_delay, len_tasks):
    global t0, count_all, last_count_all, count_success, count_empty_input,\
        count_input_too_long, count_limit_reached,\
        count_other_error, count_duplicate, count_mongo_error,\
        vocab_to_request
    try:
        
        # if empty text, directly write to db
        # else, send request and get response
        if text == '':
            count_empty_input += 1
            coll_cmt.insert_one({'id': idx, 'text_seg': '', 'text_seg2': ''})
        else:
            await asyncio.sleep(download_delay)
            # if longer than 20000 bytes, truncate
            is_too_long = False
            if len(text.encode('utf-8')) >= 20000:
                text = text.encode('utf-8')[:20000].decode('utf-8', errors='ignore')
                is_too_long = True
            async with bucket:
                    response = await get_lex(text, token)

        # process errors in response
        # if no error, generate input doc
        if 'error_code' in response:
            if response['error_code'] == 18: # limit reached
                count_limit_reached += 1
            elif response['error_code'] == 282131: # input too long
                count_input_too_long += 1
            else:
                print(f'other baidu error: request:"{text[:20]}...", response:"{response}"')
                print(token)
                count_other_error += 1
        else:
            # process successful response
            response = response['items']
            vocab_coarse = [item['item'] for item in response]
            vocab_granular = list(itertools.chain.from_iterable([item['basic_words'] for item in response]))
            text_seg = ' '.join(vocab_coarse)
            text_seg2 = ' '.join(vocab_granular)
            pos = {item['item']: item['pos'] for item in response if item['pos'] != ''}
            ne = {item['item']: item['ne'] for item in response if item['ne'] != ''}

            # write coll_text_seg_bd; coll_vocab_to_request
            with capture_output():
                coll_cmt.insert_one({'id': idx, 'text_seg': text_seg, 'text_seg2': text_seg2, 'is_too_long': is_too_long})
                count_success += 1
                for word in itertools.chain(vocab_coarse, vocab_granular):
                    if word not in vocab_to_request:
                        coll_vocab_to_request.insert_one({'word': word})
                
    except DuplicateKeyError:
        count_duplicate += 1
    except:
        print(f'Unknow Exception: {sys.exc_info()[:-1]}')
        count_other_error += 1
    finally:
    # log
        if count_all % 10000 == 0:
            print(f'{count_all}/{len_tasks}, {Now()} ({time.time()-t0:.1f}s / {(count_all-last_count_all)/(time.time()-t0):.1f}QPS)')
            print(f'success:{count_success}, limit-reached:{count_limit_reached}, empty-input-in-success:{count_empty_input}, dup:{count_duplicate}, input-too-long:{count_input_too_long}, other-baidu-error:{count_other_error}, other-mongo-error:{count_mongo_error}')
            print('---------------------')
            count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0
            t0 = time.time()
            last_count_all = count_all
    
    
async def download_many(tokens, request_cmt, coll_cmt, max_concur_req, download_delay):
    global count_all
    tokens_iter = itertools.cycle(tokens)
    len_tasks = len(request_cmt)
    # restrict concurrenty
    bucket = AsyncLeakyBucket(max_concur_req, 1)
    
    # generate tasks
    print(f'generating tasks...{Now()}')
    tasks = []
    for idx, text in request_cmt:
        task = seg_one(idx, text, next(tokens_iter), coll_cmt, bucket, download_delay, len_tasks)
        tasks.append(task)
    print(f'tasks generated {Now()}')
        
    # schedule tasks
    for future in tqdm(asyncio.as_completed(tasks), total=len_tasks):
        count_all += 1
        await future

# run the main
t0 = time.time()
count_all, last_count_all, count_success, count_empty_input, count_input_too_long, count_limit_reached, count_other_error, count_duplicate, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0, 0

asyncio.run(download_many(tokens, request_cmt_text, coll_text_seg_bd, max_concur_req=31, download_delay=0))

## ERNIE

Using ERNIE to segment is too memory expensive, give up.

In [ ]:
%%time
# infer_set = {ERNIE_LAC_PATH}/data/pred.tsv
infer_set = f'{DATA_PATH}/cmt_test_1x.tsv'

%cd {ERNIE_LAC_PATH}
!python "{ERNIE_LAC_PATH}/run_ernie_sequence_labeling.py" --ernie_config_path "{ERNIE_PATH}/pretrained/ernie_config.json" --init_pretraining_params "{ERNIE_PATH}/pretrained/params/" --init_checkpoint "{ERNIE_LAC_PATH}/model_finetuned" --init_bound 0.1 --vocab_path "{ERNIE_PATH}/pretrained/vocab.txt" --batch_size 8 --random_seed 0 --num_labels 57 --max_seq_len 64 --infer_set {infer_set} --label_map_config "{ERNIE_LAC_PATH}/conf/label_map.json" --do_lower_case true --use_cuda true --do_train false --do_test false --do_infer true
# !cd {CODE_PATH}

# Word Embedding

## create reqeust

In [ ]:
%%time
# vocab_last
vocab_with_emb = set(doc['word'] for doc in coll_emb_bd.find(projection={'word':1, '_id':0}))
print(f'vocab_with_emb: {len(vocab_with_emb)}')
   
# vocab_without_emb
vocab_without_emb = set(doc['word'] for doc in coll_vocab_without_emb.find(projection={'word':1, '_id':0}))
print(f'vocab_without_emb: {len(vocab_without_emb)}')

# delete dup in coll_vocab_to_request
vocab_to_request = set(doc['word'] for doc in coll_vocab_to_request.find(projection={'word':1, '_id':0}))
vocab_dup =  (vocab_to_request & vocab_with_emb) | (vocab_to_request & vocab_without_emb)

for word in vocab_dup:
    with capture_output():
        coll_vocab_to_request.delete_one({'word':word})

# vocab_request
vocab_request = vocab_to_request - vocab_with_emb - vocab_without_emb

# # vocab from GBK
# gbk_path = os.path.expanduser('~')+'/OneDrive/Data/gbk-list.txt'
# gbk_list = []
# with open(gbk_path, encoding='utf-8') as f:
#     for line in f:
#         line = line.replace('\n', '').strip()
#         gbk_list.append(line)
# vocab_request = set(gbk_list) - vocab_without_emb - vocab_last

print(f'vocab_request: {len(vocab_request)}')

## send requests

### sequential

In [ ]:
count_all, last_count_all, count_success, count_non_exist, count_input_too_long, count_input_empty,\
count_limit_reached, count_unknow_error, count_mongo_error, count_duplicate = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
t0 = time.time()

for word in tqdm(vocab_request):
    count_all += 1
#     time.sleep(0.1)
    try:
        response = client_bd.wordEmbedding(word)
        if 'error_code' in response:
            if response['error_code'] == 18: # limit reached
                count_limit_reached += 1
            elif response['error_code'] == 282131: # input too long / input empty
                count_input_too_long += 1
            elif response['error_code'] == 282134:
                count_input_empty += 1
            elif response['error_code'] == 282300: # non-existing
                count_non_exist += 1
                with capture_output() as captured:
                    coll_vocab_non_exist.insert_one({'word': word})
            else:
                print(f'Other error_code: word:"{word}", response:"{response}"')
                count_unknow_error += 1
        elif 'vec' in response:
            vec = response['vec']
            with capture_output() as captured:
                assert type(vec) is list, 'type(vec) is not list: {word}'
                assert len(vec) == 1024, 'len(vec) should be 1024'
                coll_embedding_bd.insert_one({'word': word, 'vec': vec})
                count_success += 1
    except DuplicateKeyError:
        count_duplicate += 1
    except:
        print(f"Exception: {sys.exc_info()[:-1]}, {Now()}")
        count_unknow_error += 1
    finally:
        if count_all % 200 == 0:
            te = time.time()
            print(f'{count_all}/{len(vocab_request)}, {Now()} ({te-t0:.1f}s / {(count_all-last_count_all)/(te-t0):.1f}QPS)')
            print(f'success:{count_success} non_existing:{count_non_exist} duplicate:{count_duplicate} limit_reached:{count_limit_reached} input_too_long:{count_input_too_long} input_empty:{count_input_empty} unkown_error:{count_unknow_error}')
            print('---------------------')
            t0 = time.time()
            count_success, count_non_exist, count_duplicate, count_input_too_long, count_input_empty,\
            count_limit_reached, count_unknow_error, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0
            last_count_all = count_all

### async

In [ ]:
async def get_emb(word, token):
    base_url = 'https://aip.baidubce.com/rpc/2.0/nlp/v2/word_emb_vec'
    url_params = {'access_token': token,
                  'charset': 'UTF-8'}
    request_text = {'word': word}
    headers = {'Content-Type': 'application/json'}

    async with aiohttp.ClientSession() as session:
        async with session.request('POST', url=base_url, json=request_text, headers=headers, params=url_params) as resp:
            response = await resp.json()
            return response

async def request_one(word, token, bucket, download_delay):
    global count_all, last_count_all, count_success,\
        count_non_exist, count_input_too_long, count_input_empty,\
        count_limit_reached, count_unknow_error,\
        count_mongo_error, count_duplicate, t0
    try:
        await asyncio.sleep(download_delay)
        async with bucket:
                response = await get_emb(word, token)
        if 'error_code' in response:
            if response['error_code'] == 18: # limit reached
                count_limit_reached += 1
            elif response['error_code'] == 282131: # input too long
                count_input_too_long += 1
            elif response['error_code'] == 282134: # input empty
                count_input_empty += 1
            elif response['error_code'] == 282300: # non-existing
                count_non_exist += 1
                with capture_output() as captured:
                    coll_vocab_without_emb.insert_one({'word': word})
            else:
                print(f'Other error_code: word:"{word}", response:"{response}"')
                count_unknow_error += 1
        elif 'vec' in response:
            vec = response['vec']
            with capture_output() as captured:
                assert type(vec) is list, 'type(vec) is not list: {word}'
                assert len(vec) == 1024, 'len(vec) should be 1024'
                coll_emb_bd.insert_one({'word': word, 'vec': vec})
                count_success += 1
    except DuplicateKeyError:
        count_duplicate += 1
    except:
        print(f"Unknow exception: {sys.exc_info()[:-1]}, {Now()}")
        count_unknow_error += 1
    finally:
        if count_all % 10000 == 0:
            te = time.time()
            print(f'{count_all}/{len(vocab_request)}, {Now()} ({te-t0:.1f}s / {(count_all-last_count_all)/(te-t0):.1f}QPS)')
            print(f'success:{count_success} non_existing:{count_non_exist} duplicate:{count_duplicate} limit_reached:{count_limit_reached} input_too_long:{count_input_too_long} input_empty:{count_input_empty} unkown_error:{count_unknow_error}')
            print('---------------------')
            t0 = time.time()
            count_success, count_non_exist, count_duplicate, count_input_too_long, count_input_empty,\
            count_limit_reached, count_unknow_error, count_mongo_error = 0, 0, 0, 0, 0, 0, 0, 0
            last_count_all = count_all
    
async def download_many(tokens, vocab_request, max_concur_req, download_delay):
    global count_all
    tokens_iter = itertools.cycle(tokens)
    # restrict concurrenty
    bucket = AsyncLeakyBucket(max_concur_req, 1)
    # generate tasks
    print(f'start creating tasks... {Now()}')
    tasks = []
    for word in vocab_request:
        task = request_one(word, next(tokens_iter), bucket, download_delay)
        tasks.append(task)
    print(f'tasks succesfully created {Now()}')
        
    # schedule tasks
    for future in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        count_all += 1
        await future

# run
t0 = time.time()

count_all, last_count_all, count_success, count_non_exist, count_input_too_long, count_input_empty, count_limit_reached, count_unknow_error, count_mongo_error, count_duplicate = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

asyncio.run(download_many(tokens, vocab_request, max_concur_req=32, download_delay=0))

# Doc Embedding

## ERNIE

### create `tsv`

- for *text*:
    - 44,405,159 original docs
    
        - 80.9% are shorter than 64

        - 89.5% are shorter than 128

        - 96.8% are shorter than 513

        - 99.2% are shorter than 2048
        
    - if split into subdocs of 128 char 
        - 53,173,056 sub-docs (max_doc_size=512)
    
        - 57,705,937 sub-docs (max_doc_size=1024)
    
        - **62,220,599 sub-docs (max_doc_size=2048)**
    
    - if split into sentences
        - 155,914,426 sentences
        
            - 64.1% shorter than 32

            - 89.9% shorter than 64
            
        - 178,013,470 sub-sents (64 char)
        
        - 249,078,800 sub-sents (32 char)
    
- for *title*:

    - 1,173,406 non-empty docs
        - 96.9% are shorter than 32 char
        
    - **1,199,615 sub-docs**
        - split level: 32 char
        
    - 1,433,301 sentences
        - split size: sentence
        
        - 98.45% are shorter than 32 char

In [67]:
def create_subdoc(cmt, cmt_type, subdoc_size, max_doc_size=None, file_size=None):
    # split every doc into chunks of size 64
    # also preserve its original row_id
    # empty docs not included
    cmt_subdoc_id = []
    cmt_subdoc = []
    for idx, doc in enumerate(cmt):
        if doc != '':
            if max_doc_size is not None:
                doc = doc[:max_doc_size]
            subdoc = list(chunks(doc, subdoc_size))
            idx = [idx] * len(subdoc)
            cmt_subdoc.extend(subdoc)
            cmt_subdoc_id.extend(idx)
    print(f'{len(cmt_subdoc)} non-empty sub-docs')
            
    for idx, file in enumerate(chunks(cmt_subdoc, file_size)):
        with open(f'../data/cmt_{cmt_type}_subdoc_{idx}.tsv', 'w', encoding='utf-8') as f:
            with capture_output():
                f.write('text_a\tlabel\n')
                for doc in file:
                    f.write(f'{doc}\t\n')
                    
    pickle.dump(cmt_subdoc_id, open(f'../data/cmt_{cmt_type}_subdoc_id.pkl', 'wb'))

In [66]:
%%time
# create test file for speed benchmarking
factor = 8
with open(f'../data/cmt_test_{factor}x.tsv', 'w', encoding='utf-8') as f:
    if 'cmt_text_clean' not in globals():
        cmt_text_clean = ld('cmt_text_clean')
    with capture_output():
        f.write('text_a\tlabel\n')
        for doc in cmt_text_clean[:20000]*factor:
            f.write(f'{doc}\t\n')

Wall time: 245 ms


In [57]:
%%time
# create title tsv
if 'cmt_title_clean' not in globals():
    cmt_title_clean = ld('cmt_title_clean')
create_subdoc(cmt_title_clean, cmt_type='title', doc_size=32, file_size=2000000)

Wall time: 3.51 s


In [69]:
%%time
# create text tsv
if 'cmt_text_clean' not in globals():
    cmt_text_clean = ld('cmt_text_clean')
create_subdoc(cmt_text_clean, cmt_type='text', subdoc_size=128, max_doc_size=2048, file_size=2000000)

62220599 non-empty sub-docs
Wall time: 2min 24s


### get emb

In [ ]:
def get_doc_emb(cmt_type, emb_level: 'subdoc or sentence', max_seq_len, batch_size, finished_file_idx):
    all_file_idx = set(int(re.search(r'\d+', str(x)).group()) for x in Path('../data').glob(f'cmt_{cmt_type}_{emb_level}_*.tsv'))
    unfinished_file_idx = all_file_idx - finished_file_idx
    
    print(f'{len(unfinished_file_idx)} file(s) to process...')
    for i, file_idx in enumerate(unfinished_file_idx):
        file_path = f'../data/cmt_{cmt_type}_{emb_level}_{file_idx}.tsv'

        print(f'{i+1}/{len(unfinished_file_idx)} (file_idx:{file_idx}) started at {Now()}')
        t0 = time.time()
        with capture_output():
            !python {ERNIE_PATH}/ernie_encoder.py --cmt_type {cmt_type} --file_idx {file_idx} --use_cuda true --batch_size {batch_size} --output_dir {DATA_PATH} --init_pretraining_params {ERNIE_PATH}/pretrained/params --data_set {file_path} --vocab_path {ERNIE_PATH}/config/vocab.txt --max_seq_len {max_seq_len} --ernie_config_path {ERNIE_PATH}/config/ernie_config.json
        print(f'{i+1}/{len(unfinished_file_idx)} finshed, use {time.time()-t0:.1f}s')
        print('-------------------------')

In [58]:
%%time
# get title
get_doc_emb(cmt_type='title',
            emb_level='subdoc',
            max_seq_len=32,
            batch_size=768,
            finished_file_idx={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16})

1 file(s) to process...
1/1 started at 15:50:06
1/1 finshed, use 1199.6s
-------------------------
Wall time: 19min 59s


In [ ]:
%%time
# get text
get_doc_emb(cmt_type='text',
            emb_level='subdoc',
            max_seq_len=128,
            batch_size=128,
            finished_file_idx={0, 1, 10, 11})

32 file(s) to process...
1/32 started at 16:46:50


## BERT

In [4]:
%%time
# sample
cmt_text_clean = ld('cmt_text_clean2')

Wall time: 37.9 s


In [7]:
%%time
# wreite cmt_text_clean
chunk_size = 1000000
n_chunk = len(cmt_text_clean)//chunk_size+1

for i in range(n_chunk):
    start = chunk_size * i
    end = start + chunk_size
    with open(f'../data/cmt_text_clean_{i}.txt', 'w', encoding='utf-8') as f:
        with capture_output():
#             f.write('text_a\tlabel\n')
            for doc in cmt_text_clean[start:end]:
                f.write(f'{doc}\n')

Wall time: 1min 13s


In [ ]:
for i in range(10):
    print(Now())
    input_file = f'cmt_text_clean_{i}'
    output_file = f'cmt_text_emb_{i}'
    
    !python {BERT_PATH}/extract_features.py --input_file={DATA_PATH}/{input_file}.txt --output_file={DATA_PATH}/{output_file}.pkl --vocab_file={BERT_PATH}/chinese_L-12_H-768_A-12/vocab.txt --bert_config_file={BERT_PATH}/chinese_L-12_H-768_A-12/bert_config.json --init_checkpoint={BERT_PATH}/chinese_L-12_H-768_A-12/bert_model.ckpt --layers=-1,-2,-3,-4 --max_seq_length=512 --batch_size=16
    print(Now())

In [1]:
x = ld('output')
len(x)

10000